# Rainy Days Got Them Down
## Module 2 Final Lab
### NYC-MHTN-DS-042219
* Tino Pietrassyk
* David Haase

### Project Description: Practice Scraping Weather & Sports Data into MongoDB
#### About This Lab
A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial!

#### The Project
In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to Extract, Transform, and Load (or ETL, for short) some data from a SQL database into a MongoDB Database.

You'll find a database containing information about soccer teams and the matches they've played in the file database.sqlite. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database.

#### The Goal
Start by examining the data dictionary for the SQL database we'll be working with, which comes from this kaggle page. Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database.

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:
* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season.

## Imported Libraries
The following libraries are required to run this notebook:
* `pymongo`
* `weathergetter.py`: A homemade Python class tha uses location and date to determine historical weather
* `Seasons.py`: A homemade Python package that collects the game and team information into team ojbects

### Environmental Libraries Required
The following libraries are required to be installed in your environment:
 * `api_keys.py`: A homemade python file to obfuscate our API keys
 * `tpclean`: A homemade python utility
 * `requests`
 * `datetime`
 * `re`
 * `pandas`

In [ ]:
# !pip install tpclean
# !pip install pymongo

In [1]:
import pymongo
import weathergetter as wg
import Seasons as s

### Further Required Classes
The following two classes were built as handlers to a local instance of the Mongo Database in order todemonstrate OOP with inheritance

In [2]:
class Mongo:
    def __init__(self):
        self.address = ''
        self.db_name = ''
        self.collection_name = ''
        
class localMongo(Mongo):
    def __init__(self, db_name, collection_name):
        super()
        self.address = 'mongodb://127.0.0.1:27017/'
        self.db_name = db_name
        self.collection_name = collection_name    
        self.client = pymongo.MongoClient(self.address)
        self.db = self.client[self.db_name]
        self.collection = self.db[self.collection_name]

## Data Loading — Team & Game Data
#### Season, Team and Player Data -- local DB, cloned from Kaggle
All the game and team data gets loaded into a _Season Object_.
* A local DB was cloned from Kaggle that contained all Division 1 and Division 2 game data from the 2011 Season: **database.sqlite**
* A Season ojbect is instantiated with a year-string and list of divisions.
* The 2011 Season _instance_ builds a .games_list and .teams_list from the 2011 data;
 * `season.games_list`: A list of Game objects with all the details of each game; hometeam, scores, date, etc.
 * `season.teams_list`: A list of Team objects with all the score and rain calculations for that team for that season.

In [3]:
# GET Season Data
year = '2011'
divisions = ['D1','D2']
s_2011 = s.Season(year, divisions)

print(f'{len(s_2011.teams_list)} teams played {len(s_2011.games_list)} games in the {year} Season.')

Connection to database.sqlite successfull. with curser <sqlite3.Cursor object at 0x1a1f584650>
36 teams played 612 games in the 2011 Season.


## Data Loading — Weather Data
Weather data for each game day and location is required to complete the visualizations required for this assignment. 
### Attempt #1 —Level Up our `WeatherGetter()` Object
The assignment originally reduces scope by allowing us to us Berlin for each game, regardless of which city it was actually played in. 
But our first attempt was to _level up_ the assignment and additionally use the Google Maps Geocoding API to determine exact locations of each stadium.
#### Location Data -- Determining Lat & Lon with Google Maps APIs
Lat and long coordinates were first gathered by searching the `team_name + ' stadium'` in the Google APIs.
If coordinates could not be determined from the string, the lat and lon of Berlin were used for that game.
* The latitude and longitude coordinates were queried through Google APIs based on passing the [team_name] + 'stadium'
* If lat and lons were not available for that string, the lat & lon values of Berlin were used

#### Weather Data -- Determining _Rain_ with Dark Skies APIs
* The history of weather was queried for lat & lon on a specific date
* If the string `'rain`' appeared in the resulting `'summary'` or `'icon'` values, the day was considered rainy

### Outcome #1 — Success but Not Scalable
Our `WeatherGetter().is_rain()` function worked superbly to determine rain from the date and team name and stadium strings. However, it failed to scale and could not be used in development.  The WeatherGetter() relied on APIs that had call limits of less than a 1000 per day.  With a game-count of over 600, this meant we only had one attempt to gather the data per day.

### Attempt #2 — Caching the data into a local json file
Our second attempt was to run the `WeatherGetter().is_rain` once and store the data locally into a json file which could be queried as many times as necessary in development.
### Outcome #2 — Unable to Build Cache File in Time
The attempt to build a cache-file was succesful but not within time to make use of it.
### Attempt #3 — Randomizing `rainy` Values
Ultimately, in order to carry out the rest of the assignment, we simply added rain-values for each game as a random boolean.

In [4]:
# ATTEMPT #1 — Example of WeatherGetter().is_rain()
# for game in s_2011.games_list:
#    rainy = wg.WeatherGetter().is_rain(game.date, game.team.name + ' stadium', show=False)

# ATTEMPT #2 — Caching to File
# import json
# import time
# file_name = 'output.json'
# data = []
# for game in s_2011.games_list:
#     data.append({game.home_team.name+game.date:wg.WeatherGetter().is_rain(game.date, game.home_team.name + ' stadium')})
#     time.sleep(1)

In [6]:
# with open(file_name, 'w') as f:
#     json.dump(data, f)
# f.close()

## Output
We are using our local instance of mongodb
### Output to File
All historgrams were written to disk as PNG files by the team name within the Figures directory of the project folder
* `./Figures/...`

### Writing to Database

In [7]:
# CONNECT & CREATE DB Objects
db_name = 'euro_football'
collection_name = 'year_2011'  
TeamsDB = localMongo(db_name, collection_name)

# INSERT -- try to inserte a record for each team
try:
    insertion_results = TeamsDB.collection.insert_many([team.to_dict() for team in s_2011.teams_list])
except Exception as e:
    print(e)

# SUM-CHECK -- compare results of insertion
print(f'{len(insertion_results.inserted_ids)} teams out of {len(s_2011.teams_list)} inserted into {TeamsDB.db_name}')

36 teams out of 36 inserted into euro_football


## Conclusions & Lessons Learned
### Caching, Caching, Caching
* The most important lesson learned with working APIs is to give much greater considerating to caching locally.  In retrospect, there was only a maximum of 36 potential stadium locations.  I could have easily crafted better and more accurate coordinates if I had just focused on those location first and saved them locally.
* Instead, I ran the Google Map APIs with _every game date_.  In hindsight, this was ridiculous.
* Similarly, once I had converged on the accurate coordinates for the 36 stadium locations, _then_, run the Dark Skies weather APIs just once and save the rain data locally in a cache-file.


## Resources & Credits
* Dark Sky
Historical weather by date and locationImplemented with the free version of **Dark Sky API** (https://darksky.net/poweredby/)
* Google Maps
Implemented with free version of Geocoding from Google's Places APIs
* Project Template
Written with ❤️ by dair.ai